In [11]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [12]:
sentences = ['How may I help you?',
             'Can I be of assistance?',
             'May I help you with something?',
             'May I assist you?']

In [20]:
words = dict()
reverse = dict()
i = 0
for s in sentences:
    s = s.replace('?',' <unk>')
    for w in s.split():
        if w.lower() not in words:
            words[w.lower()] = i
            reverse[i] = w.lower()
            i = i + 1

In [21]:
class DataGenerator():
    def __init__(self, dset):
        self.dset = dset
        self.len = len(self.dset)
        self.idx = 0
    def __len__(self):
        return self.len
    def __iter__(self):
        return self
    def __next__(self):
        x = Variable(torch.LongTensor([[self.dset[self.idx]]]), requires_grad=False)
        if self.idx == self.len - 1:
            raise StopIteration
        y = Variable(torch.LongTensor([self.dset[self.idx+1]]), requires_grad=False)
        self.idx = self.idx + 1
        return (x, y)

In [31]:
class BotBrain(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding = nn.Embedding(len(words), 10)
        self.rnn = nn.LSTM(10, 20, 2, dropout=0.5)
        self.h = (Variable(torch.zeros(2, 1, 20)), Variable(torch.zeros(2, 1, 20)))
        self.l_out = nn.Linear(20, len(words))
        
    def forward(self, cs):
        inp = self.embedding(cs)
        outp,h = self.rnn(inp, self.h)
        out = F.log_softmax(self.l_out(outp), dim=-1).view(-1, len(words))
        return out

In [32]:
m = BotBrain()

In [33]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(m.parameters(), lr=0.01)
for epoch in range(0,300):
    gen = DataGenerator([words[word.lower()] for word in ' '.join(sentences).replace('?',' <unk>').split(' ')])
    for x, y in gen:
        m.zero_grad()
        output = m(x)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
print(loss)

Variable containing:
 1.6654
[torch.FloatTensor of size 1]



In [ ]:
def get_next(word_):
    word = word_.lower()
    out = m(Variable(torch.LongTensor([words[word_]])))
    return reverse[int(out.max(dim=1)[1].data)]

In [ ]:
def get_next_n(word_, n=3):
    print(word_)
    for i in range(0, n):
        word_ = get_next(word_)
        print(word_)

In [ ]:
get_next_n('<unk>', n=12)